In [1]:
%load_ext autoreload
%autoreload 1
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import timm
from timm import optim, scheduler
import torch
from torchvision import transforms as tfm
from sklearn import metrics as skmet
import matplotlib.pyplot as plt
import json
import transforms as my_transforms

%aimport dataset
ImageData = dataset.ImageData

In [3]:
# artifact_folder = '/zfs/wficai/pda/model_run_artifacts/20220818_justcolor_224x224'
artifact_folder = '/zfs/wficai/pda/model_run_artifacts/20220818_no2d_224x224'

with open(artifact_folder + '/config.json', 'r') as f: 
    cfg = json.load(f)

# put all config variables in scope to avoid the need to laboriously index cfg
for k, v in cfg.items():
    v = f"'{v}'" if type(v)==str else v
    exec(f"{k}={v}")
del cfg

In [4]:
# optionally override settings
view_filter = ['pdaView', 'pdaRelatedView']
mode_filter = ['2d', 'color', 'color_compare']
device = torch.device('cuda:1')  # you may need 'cuda:0'

In [5]:
tfms = my_transforms.ImageTransforms(res)
tfms_test = tfms.get_transforms(transforms['test'])

In [6]:
df_test = pd.read_csv(f'{artifact_folder}/{out_paths["test"]}')
d_test = ImageData(df_test, transforms = tfms_test, mode_filter = mode_filter, view_filter = view_filter)
dl_test = DataLoader(d_test, batch_size=bs_test, num_workers=num_workers)

print("Number of frames after filtering:", len(d_test.data))

Number of frames after filtering: 14293


In [7]:
# create model
m = timm.create_model(model, pretrained=pretrained, checkpoint_path = f"{artifact_folder}/model_checkpoint.ckpt", num_classes=num_classes, in_chans=3)
m.to(device)           
m.eval()

loss_function = torch.functional.F.binary_cross_entropy_with_logits

target_ls = []
output_ls = []
study_ls = []
video_ls = []
view_ls = []
mode_ls = []
losses = []

for ix, batch in enumerate(dl_test):
    print(f"Batch {ix+1}", end = "\r")
    inputs = batch['img'].to(device)
    targets = batch['trg_type'].to(device).type(torch.float32)
    target_ls.append(targets.cpu().numpy())
    view_ls.append(batch['trg_view'].numpy())
    mode_ls.append(batch['trg_mode'].numpy())
    study_ls += batch['study']
    video_ls += batch['video']

    with torch.no_grad():
        outputs = m(inputs)
        output_ls.append(outputs.cpu().numpy())
        loss = loss_function(outputs.squeeze(), targets)
        losses.append(loss.detach().item())

In [8]:
df_results = pd.DataFrame(dict(
    study = study_ls,
    video = video_ls,
    predicted = np.concatenate(output_ls).squeeze(),
    target = np.concatenate(target_ls), 
    mode = np.concatenate(mode_ls),
    view = np.concatenate(view_ls)
))

df_results['mode'] = df_results['mode'].map(ImageData.inv_mode_map)
df_results.view = df_results.view.map(ImageData.inv_view_map)

df_results.predicted = 1 / (1 + np.exp(-df_results.predicted))

df_results.head(20)

,study,video,predicted,target,mode,view
0,study57,study57_dicom61,0.638757,0.0,color,pdaView
1,study57,study57_dicom61,0.460421,0.0,color,pdaView
2,study57,study57_dicom61,0.613172,0.0,color,pdaView
3,study57,study57_dicom61,0.440403,0.0,color,pdaView
4,study57,study57_dicom61,0.582152,0.0,color,pdaView
5,study57,study57_dicom61,0.508617,0.0,color,pdaView
6,study57,study57_dicom61,0.491698,0.0,color,pdaView
7,study57,study57_dicom61,0.557328,0.0,color,pdaView
8,study57,study57_dicom61,0.593047,0.0,color,pdaView
9,study57,study57_dicom61,0.737592,0.0,color,pdaView


In [9]:
def compute_metrics(y_true, y_pred, thresh=0.5):
    mets = dict()
    
    y_pred_cls = (y_pred>thresh).astype(int)
    
    mets['num_samples'] = len(y_true)
    mets['roc_auc'] = skmet.roc_auc_score(y_true, y_pred)
    mets['average_precision'] = skmet.average_precision_score(y_true, y_pred)
    mets['accuracy'] = skmet.accuracy_score(y_true, y_pred_cls)
    mets['sensitivity'] = skmet.recall_score(y_true, y_pred_cls)
    mets['specificity'] = skmet.recall_score(y_true, y_pred_cls, pos_label=0)
    
    return mets

# Frame-level results

In [10]:
compute_metrics(df_results.target, df_results.predicted)

{'num_samples': 14293,
 'roc_auc': 0.8376700943507704,
 'average_precision': 0.8974763330859286,
 'accuracy': 0.7247603722101729,
 'sensitivity': 0.9471846846846846,
 'specificity': 0.35987437650101606}

In [11]:
grouped_results = df_results.groupby(['view', 'mode']).apply(lambda dat: compute_metrics(dat.target, dat.predicted))
grouped_results = pd.DataFrame(grouped_results.tolist(), index=grouped_results.index)
grouped_results

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                 4062  0.843871           0.816142   
               color_compare         2877  0.719499           0.851422   
pdaView        color                 2297  0.866595           0.906748   
               color_compare         5057  0.863768           0.946314   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.639833     0.956498     0.383793  
               color_compare  0.695864     0.872951     0.322162  
pdaView        color          0.717893     0.955994     0.397344  
               color_compare  0.812537     0.977860     0.315914

# Clip-level results

### Avg confidence over frames

In [12]:
df_results_clip_avg = df_results.groupby(['study', 'video', 'target', 'view', 'mode'], as_index=False).agg('mean')
display(df_results_clip_avg.head(10))

compute_metrics(df_results_clip_avg.target, df_results_clip_avg.predicted)

,study,video,target,view,mode,predicted
0,study1,study1_dicom30,0.0,pdaRelatedView,color,0.526393
1,study1,study1_dicom55,0.0,pdaRelatedView,color,0.571489
2,study11,study11_dicom36,0.0,pdaView,color_compare,0.587726
3,study11,study11_dicom49,0.0,pdaView,color_compare,0.554994
4,study12,study12_dicom17,1.0,pdaRelatedView,color,0.718332
5,study12,study12_dicom21,1.0,pdaView,color,0.797686
6,study12,study12_dicom24,1.0,pdaView,color_compare,0.721429
7,study12,study12_dicom25,1.0,pdaView,color_compare,0.707604
8,study13,study13_dicom11,1.0,pdaRelatedView,color_compare,0.739857
9,study13,study13_dicom3,1.0,pdaRelatedView,color,0.785980


{'num_samples': 142,
 'roc_auc': 0.9376016260162602,
 'average_precision': 0.9623687335371922,
 'accuracy': 0.6408450704225352,
 'sensitivity': 1.0,
 'specificity': 0.15}

In [13]:
grouped_results_clip_avg = df_results_clip_avg.\
    groupby(['view', 'mode']).\
    apply(lambda dat: compute_metrics(dat.target, dat.predicted, thresh=0.55))
grouped_results_clip_avg = pd.DataFrame(grouped_results_clip_avg.tolist(), index=grouped_results.index)
grouped_results_clip_avg

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                   54  0.936111           0.934164   
               color_compare           27  0.852941           0.929313   
pdaView        color                   24  0.971429           0.984127   
               color_compare           37  0.988889           0.996074   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.666667     1.000000          0.4  
               color_compare  0.666667     0.882353          0.3  
pdaView        color          0.750000     1.000000          0.4  
               color_compare  0.756757     1.000000          0.1

### Misses

In [14]:
true = df_results_clip_avg.target
pred_cls = (df_results_clip_avg.predicted>0.5).astype(int)

df_results_clip_avg[true != pred_cls]

,study,video,target,view,mode,predicted
0,study1,study1_dicom30,0.0,pdaRelatedView,color,0.526393
1,study1,study1_dicom55,0.0,pdaRelatedView,color,0.571489
2,study11,study11_dicom36,0.0,pdaView,color_compare,0.587726
3,study11,study11_dicom49,0.0,pdaView,color_compare,0.554994
22,study21,study21_dicom28,0.0,pdaRelatedView,color,0.551039
23,study21,study21_dicom45,0.0,pdaRelatedView,color,0.711959
31,study23,study23_dicom64,0.0,pdaRelatedView,color,0.651448
32,study23,study23_dicom67,0.0,pdaRelatedView,color,0.566884
33,study23,study23_dicom75,0.0,pdaRelatedView,color_compare,0.597282
34,study23,study23_dicom76,0.0,pdaRelatedView,color_compare,0.566655


# Study-level results

In [15]:
df_results_study_avg = df_results_clip_avg.groupby(['study', 'target'], as_index=False).agg('mean')
display(df_results_study_avg)

compute_metrics(df_results_study_avg.target, df_results_study_avg.predicted)

,study,target,predicted
0,study1,0.0,0.548941
1,study11,0.0,0.571360
2,study12,1.0,0.736263
3,study13,1.0,0.762758
4,study16,1.0,0.786605
5,study2,1.0,0.760370
6,study21,0.0,0.631499
7,study22,1.0,0.791374
8,study23,0.0,0.595567
9,study26,1.0,0.709062


{'num_samples': 31,
 'roc_auc': 0.9956140350877194,
 'average_precision': 0.9935897435897437,
 'accuracy': 0.41935483870967744,
 'sensitivity': 1.0,
 'specificity': 0.05263157894736842}